In [196]:
%pylab qt
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


In [197]:

root_symbol='a'
value_name='alpha'
root_name=f'{value_name}-{root_symbol}'

directory_pattern = f'^{root_name}.*'
value_pattern = f'.*{root_symbol}(-?\d+.\d+)(pi)?'

In [198]:
#get a dataframe containing the energies

dirlist = os.listdir()
data_paths = []
energies_df=pd.DataFrame(columns=['alpha_x', 'Tot.E'])
for directory in dirlist:
    if os.path.isdir(directory) and re.search(directory_pattern, directory):
        print(directory)
        alpha = float(re.match(value_pattern, directory).group(1))
        print(alpha)
        energy_filepath = os.path.join(directory, 'etot.pl')
        
        columns=['Time', 'Energy', 'Norm']
        single_energy_pd=pd.read_csv(energy_filepath, names=columns, delim_whitespace=True)
        average_e = single_energy_pd['Energy'].mean()
        
        energies_df=energies_df.append({'alpha_x': alpha, 'Tot.E':average_e}, ignore_index=True)

#energies_df=energies_df.sort_values(by='Momentum', ascending=True)  
energies_df.sort_values(by='alpha_x', ascending=True, inplace=True)
energies_df

alpha-a-0.075pi
-0.075
alpha-a-0.125pi
-0.125
alpha-a-0.25pi
-0.25
alpha-a-0.375pi
-0.375
alpha-a-0.4pi
-0.4
alpha-a0.075pi
0.075
alpha-a0.0pi
0.0
alpha-a0.125pi
0.125
alpha-a0.25pi
0.25
alpha-a0.375pi
0.375
alpha-a0.4pi
0.4


,alpha_x,Tot.E
4,-0.400,9.001814
3,-0.375,9.001814
2,-0.250,9.001814
1,-0.125,9.001814
0,-0.075,9.001814
6,0.000,9.001814
5,0.075,8.660646
7,0.125,5.226400
8,0.250,9.001814
9,0.375,9.001814


In [199]:
dirlist = os.listdir()
adp_paths = []
for directory in dirlist:
    if os.path.isdir(directory) and re.search(directory_pattern, directory):
        alpha = float(re.match(value_pattern, directory).group(1))
        adp_filepath = os.path.join(directory, 'adp-a{}'.format(alpha))
        adp_paths.append(adp_filepath)
        
adp_paths

['alpha-a-0.075pi\\adp-a-0.075',
 'alpha-a-0.125pi\\adp-a-0.125',
 'alpha-a-0.25pi\\adp-a-0.25',
 'alpha-a-0.375pi\\adp-a-0.375',
 'alpha-a-0.4pi\\adp-a-0.4',
 'alpha-a0.075pi\\adp-a0.075',
 'alpha-a0.0pi\\adp-a0.0',
 'alpha-a0.125pi\\adp-a0.125',
 'alpha-a0.25pi\\adp-a0.25',
 'alpha-a0.375pi\\adp-a0.375',
 'alpha-a0.4pi\\adp-a0.4']

In [200]:
def plot_diabatic_qdq(dof='x', state=2, supress_plot=False):
    
    #Get paths of all expectation value files, append to data_paths
    dirlist = os.listdir()
    data_paths = []
    for directory in dirlist:
        if os.path.isdir(directory) and re.search(directory_pattern, directory):
            path = os.path.join(directory, 'expectations_dir')
            datafiles = os.listdir(path)
            for file in datafiles:
                data_path = os.path.join(path, file)
                data_paths.append(data_path)

    #Load expectation data into python using pandas
    q_all=[]
    q2_all=[]
    for path in data_paths:
        #get just the filename for regex
        filename = os.path.split(path)[1]
        if re.search(r'^{0}diabstate{1}.*'.format(dof, state), filename):
            print(path)
            #find starting momentum from filename using regex
            alpha = float(re.match(value_pattern, filename).group(1))
            print(alpha)
            #create dataframe
            columns = ['Time', '<{}>'.format(dof), 'imaginary', 'norm']
            q_entry = pd.read_csv(path, skiprows=0, names=columns, delim_whitespace=True, comment='#')
            #This part is to add starting momentum labels so we can have a multi-index dataframe later
            tilt_labels = np.ones(len(q_entry))*float(alpha)
            q_entry['init alpha']=tilt_labels
            q_entry = q_entry.set_index(['init alpha', np.arange(0, len(q_entry))])
            q_all.append(q_entry.drop(['imaginary', 'norm'], axis=1))    
        
        # Match files containing <q^2>    
        if re.search('^{0}2diabstate{1}.*'.format(dof, state), filename):
            print(filename)
            #find starting momentum from filename using regex
            alpha = float(re.match(value_pattern, filename).group(1))
            #create dataframe
            columns = ['Time', '<{}^2>'.format(dof), 'imaginary', 'norm']
            q2_entry = pd.read_csv(path, skiprows=0, names=columns, delim_whitespace=True, comment='#')
            #This part is to add starting momentum labels so we can have a multi-index dataframe later
            tilt_labels = np.ones(len(q2_entry))*float(alpha)
            q2_entry['init alpha']=tilt_labels
            q2_entry = q2_entry.set_index(['init alpha', np.arange(0, len(q2_entry))])
            q2_all.append(q2_entry.drop(['imaginary', 'norm'], axis=1))
    
    
    
    if len(q_all)>1:
        q_df = pd.concat(q_all).sort_index()
        q2_df = pd.concat(q2_all).sort_index()
    else:
        q_df = q_all[0]
        q2_df = q2_all[0]
        
    qdq_df = q_df.join(q2_df.drop(['Time'], axis=1))
    
    qdq_df['<d{}>'.format(dof)] = np.sqrt(qdq_df['<{}^2>'.format(dof)] - qdq_df['<{}>'.format(dof)]**2)
    
    if supress_plot:
        return qdq_df
    
    
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    fig.figsize = (10, 8)
    for alpha, entry in qdq_df.groupby(level=0):
        single_tilt_data = entry.droplevel(0)
        ax1.plot(single_tilt_data['Time'], single_tilt_data['<{}>'.format(dof)], 
                label='{}'.format(alpha), linestyle='-', marker='', markersize=1)
        ax1.set_title('Expectation values for DOF {0} on diabatic state {1}'.format(dof, state))
        ax2.plot(single_tilt_data['Time'], single_tilt_data['<d{}>'.format(dof)], 
                linestyle='-', marker='', markersize=1)

  
    ax2.set_xlabel('Time /fs')
    ax1.set_ylabel('<{}>'.format(dof))
    ax2.set_ylabel('<d{}>'.format(dof))
    fig.legend(title='Alpha /radians')
    fig.show()
    
    return qdq_df

In [201]:
qdq_all = plot_diabatic_qdq(dof='x', state=2)


x2diabstate2-a-0.23561944901923448
alpha-a-0.075pi\expectations_dir\xdiabstate2-a-0.23561944901923448
-0.23561944901923448
x2diabstate2-a-0.39269908169872414
alpha-a-0.125pi\expectations_dir\xdiabstate2-a-0.39269908169872414
-0.39269908169872414
x2diabstate2-a-0.7853981633974483
alpha-a-0.25pi\expectations_dir\xdiabstate2-a-0.7853981633974483
-0.7853981633974483
x2diabstate2-a-1.1780972450961724
alpha-a-0.375pi\expectations_dir\xdiabstate2-a-1.1780972450961724
-1.1780972450961724
x2diabstate2-a-1.2566370614359172
alpha-a-0.4pi\expectations_dir\xdiabstate2-a-1.2566370614359172
-1.2566370614359172
x2diabstate2-a0.23561944901923448
alpha-a0.075pi\expectations_dir\xdiabstate2-a0.23561944901923448
0.23561944901923448
x2diabstate2-a0.0
alpha-a0.0pi\expectations_dir\xdiabstate2-a0.0
0.0
x2diabstate2-a0.39269908169872414
alpha-a0.125pi\expectations_dir\xdiabstate2-a0.39269908169872414
0.39269908169872414
x2diabstate2-a0.7853981633974483
alpha-a0.25pi\expectations_dir\xdiabstate2-a0.78539816339

In [202]:
qdq_all['Time'].iloc[-60]

33.0

In [203]:
def plot_adiabatic_qdq(dof='x', state=2, supress_plot=False):
    
    dirlist = os.listdir()
    data_paths = []
    for directory in dirlist:
        if os.path.isdir(directory) and re.search(directory_pattern, directory):
            path = os.path.join(directory, 'expectations_dir')
            datafiles = os.listdir(path)
            for file in datafiles:
                data_path = os.path.join(path, file)
                data_paths.append(data_path)
                
    q_all=[]
    q2_all=[]
    for path in data_paths:
        filename = os.path.split(path)[1]
        # Match files containing <q>
        if re.search(r'^{0}adbstate{1}.*'.format(dof, state), filename):
            #find starting momentum from filename using regex
            alpha = float(re.match(value_pattern, filename).group(1))
            
            #create dataframe
            columns = ['Time', '<{}>'.format(dof), 'imaginary', 'norm']
            q_entry = pd.read_csv(path, skiprows=0, names=columns, delim_whitespace=True, comment='#')
            #This part is to add starting momentum labels so we can have a multi-index dataframe later
            tilt_labels = np.ones(len(q_entry))*float(alpha)
            q_entry['init alpha']=tilt_labels
            q_entry = q_entry.set_index(['init alpha', np.arange(0, len(q_entry))])
            q_all.append(q_entry.drop(['imaginary', 'norm'], axis=1))    
        
        # Match files containing <q^2>    
        if re.search('^{0}2adbstate{1}.*'.format(dof, state), filename):
            
            #find starting momentum from filename using regex
            alpha = float(re.match(value_pattern, filename).group(1))
            
            #create dataframe
            columns = ['Time', '<{}^2>'.format(dof), 'imaginary', 'norm']
            q2_entry = pd.read_csv(path, skiprows=0, names=columns, delim_whitespace=True, comment='#')
            #This part is to add starting momentum labels so we can have a multi-index dataframe later
            tilt_labels = np.ones(len(q2_entry))*float(alpha)
            q2_entry['init alpha']=tilt_labels
            q2_entry = q2_entry.set_index(['init alpha', np.arange(0, len(q2_entry))])
            q2_all.append(q2_entry.drop(['imaginary', 'norm'], axis=1))  
    
    
    if len(q_all)>1:
        q_df = pd.concat(q_all).sort_index()
        q2_df = pd.concat(q2_all).sort_index()
    else:
        q_df = q_all[0]
        q2_df = q2_all[0]
        
    print(q2_df)
    print(q_df)
    qdq_df = q_df.join(q2_df.drop(['Time'], axis=1))
    
    qdq_df['<d{}>'.format(dof)] = np.sqrt(qdq_df['<{}^2>'.format(dof)] - qdq_df['<{}>'.format(dof)]**2)
    
    if supress_plot:
        return qdq_df
    
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
    fig.figsize = (10, 8)
    ax1.axhline(y=0, color='grey', linestyle='--')
    for alpha, entry in qdq_df.groupby(level=0):
        single_tilt_data = entry.droplevel(0)
        ax1.plot(single_tilt_data['Time'], single_tilt_data['<{}>'.format(dof)], 
                label='{}'.format(alpha), linestyle='-', marker='', markersize=1)
        ax1.set_title('Expectation values for DOF {0} on adiabatic state {1}'.format(dof, state))
        ax2.plot(single_tilt_data['Time'], single_tilt_data['<d{}>'.format(dof)], 
                linestyle='-', marker='', markersize=1)

  
    ax2.set_xlabel('Time /fs')
    ax1.set_ylabel('<{}>'.format(dof))
    ax2.set_ylabel('<d{}>'.format(dof))
    fig.legend(title='Alpha /radians')
    fig.show()
    
    return qdq_df

In [204]:
matplotlib.rcParams.update({'font.size': 15})

In [214]:
plot_adiabatic_qdq(dof='x', state=2)

               Time      <x^2>
init alpha                    
-1.256637  0    0.0   3.437271
           1    1.5   2.900214
           2    3.0   2.293553
           3    4.5   1.665792
           4    6.0   1.075162
...             ...        ...
 1.256637  36  54.0  24.263965
           37  55.5  29.500804
           38  57.0  35.436417
           39  58.5  42.120081
           40  60.0  49.602611

[451 rows x 2 columns]
               Time       <x>
init alpha                   
-1.256637  0    0.0 -1.768944
           1    1.5 -1.613471
           2    3.0 -1.417634
           3    4.5 -1.181622
           4    6.0 -0.906723
...             ...       ...
 1.256637  36  54.0 -3.637228
           37  55.5 -4.010730
           38  57.0 -4.395887
           39  58.5 -4.792716
           40  60.0 -5.201350

[451 rows x 2 columns]


Time       <x>      <x^2>      <dx>
init alpha                                        
-1.256637  0    0.0 -1.768944   3.437271  0.555076
           1    1.5 -1.613471   2.900214  0.544910
           2    3.0 -1.417634   2.293553  0.532791
           3    4.5 -1.181622   1.665792  0.519192
           4    6.0 -0.906723   1.075162  0.503006
...             ...       ...        ...       ...
 1.256637  36  54.0 -3.637228  24.263965  3.321828
           37  55.5 -4.010730  29.500804  3.662629
           38  57.0 -4.395887  35.436417  4.014049
           39  58.5 -4.792716  42.120081  4.376066
           40  60.0 -5.201350  49.602611  4.748533

[451 rows x 4 columns]

In [215]:
def plot_adiabatic_momenta(dof='x', state=2, supress_plot=False):
    
    dirlist = os.listdir()
    data_paths = []
    for directory in dirlist:
        if os.path.isdir(directory) and re.search(directory_pattern, directory):
            path = os.path.join(directory, 'expectations_dir')
            datafiles = os.listdir(path)
            for file in datafiles:
                data_path = os.path.join(path, file)
                data_paths.append(data_path)
                
    q_all=[]
    for path in data_paths:
        filename = os.path.split(path)[1]
        # Match files containing <q>
        if re.search(r'p_{0}adbstate{1}.*'.format(dof, state), filename):
            #find starting momentum from filename using regex
            alpha = float(re.match(value_pattern, filename).group(1))
            
            #create dataframe
            columns = ['Time', '<p_{}>'.format(dof), 'imaginary', 'norm']
            q_entry = pd.read_csv(path, skiprows=0, names=columns, delim_whitespace=True, comment='#')
            #This part is to add starting momentum labels so we can have a multi-index dataframe later
            tilt_labels = np.ones(len(q_entry))*float(alpha)
            q_entry['alpha']=tilt_labels
            q_entry = q_entry.set_index(['alpha', np.arange(0, len(q_entry))])
            q_all.append(q_entry.drop(['imaginary', 'norm'], axis=1))    
        
    
    
    if len(q_all)>1:
        q_df = pd.concat(q_all).sort_index()
    else:
        q_df = q_all[0]
        
    
    
    
    if supress_plot:
        return q_df
    
    fig, ax1 = plt.subplots(1, 1)
    fig.figsize = (10, 8)
    for alpha, entry in q_df.groupby(level=0):
        single_alpha_data = entry.droplevel(0)
        ax1.plot(single_alpha_data['Time'], single_alpha_data['<p_{}>'.format(dof)], 
                label='{}'.format(alpha), linestyle='-', marker='', markersize=1)
        ax1.set_title('Expectation values for DOF {0} on adiabatic state {1}'.format(dof, state))
       
  
    ax1.set_xlabel('Time /fs')
    ax1.set_ylabel('$<p_{}>$ /a.u.'.format(dof))
    fig.legend(title='Alpha:')
    fig.show()
    
plot_adiabatic_momenta(state=1)

In [216]:
plot_adiabatic_momenta()

In [217]:
def plot_adiabatic_KE(dof='x', state=2, supress_plot=False):
    
    dirlist = os.listdir()
    data_paths = []
    for directory in dirlist:
        if os.path.isdir(directory) and re.search(directory_pattern, directory):
            path = os.path.join(directory, 'expectations_dir')
            datafiles = os.listdir(path)
            for file in datafiles:
                data_path = os.path.join(path, file)
                data_paths.append(data_path)
                
    q_all=[]
    for path in data_paths:
        filename = os.path.split(path)[1]
        # Match files containing <q>
        if re.search(r'KE_{0}adbstate{1}.*'.format(dof, state), filename):
            #find starting momentum from filename using regex
            alpha = float(re.match(value_pattern, filename).group(1))
            
            #create dataframe
            columns = ['Time', '<KE_{}>'.format(dof), 'imaginary', 'norm']
            q_entry = pd.read_csv(path, skiprows=0, names=columns, delim_whitespace=True, comment='#')
            #This part is to add starting momentum labels so we can have a multi-index dataframe later
            tilt_labels = np.ones(len(q_entry))*float(alpha)
            q_entry['alpha']=tilt_labels
            q_entry = q_entry.set_index(['alpha', np.arange(0, len(q_entry))])
            q_all.append(q_entry.drop(['imaginary', 'norm'], axis=1))    
        
    
    
    if len(q_all)>1:
        q_df = pd.concat(q_all).sort_index()
    else:
        q_df = q_all[0]
        
    
    
    
    if supress_plot:
        return q_df
    
    fig, ax1 = plt.subplots(1, 1)
    fig.figsize = (10, 8)
    for alpha, entry in q_df.groupby(level=0):
        single_alpha_data = entry.droplevel(0)
        ax1.plot(single_alpha_data['Time'], single_alpha_data['<KE_{}>'.format(dof)], 
                label='{}'.format(alpha), linestyle='-', marker='', markersize=1)
        ax1.set_title('Expectation values for DOF {0} on adiabatic state {1}'.format(dof, state))
       
  
    ax1.set_xlabel('Time /fs')
    ax1.set_ylabel('$<KE_{}>$ /a.u.'.format(dof))
    fig.legend(title='Alpha:')
    fig.show()
    
plot_adiabatic_KE()

In [218]:
def plot_adpop(representation='adb', state=2):
    
    dirlist = os.listdir()
    adp_paths = []
    for directory in [path for i, path in enumerate(dirlist) if os.path.isdir(dirlist[i])]:
        in_dir = os.listdir(directory)
        for file in in_dir:
            if re.search('adp-a.*', file):
                adp_filepath = os.path.join(directory, file)
                adp_paths.append(adp_filepath)
            
            
    print(adp_paths) 
    
    adpop_all=[]
    for path in adp_paths:
        filename=os.path.split(path)[1]
        if re.search('^adp-', filename):
            print(filename)
            #find starting momentum from filename using regex
            alpha = float(re.match(value_pattern, filename).group(1))
            
            #create dataframe
            columns = ['Time', 'a1', 'a2', 'd1', 'd2']
            adpop_entry = pd.read_csv(path, skiprows=0, names=columns, delim_whitespace=True, comment='#')
            #This part is to add starting momentum labels so we can have a multi-index dataframe later
            tilt_labels = np.ones(len(adpop_entry))*float(alpha)
            adpop_entry['alpha']=tilt_labels
            adpop_entry = adpop_entry.set_index(['alpha', np.arange(0, len(adpop_entry))])
            adpop_all.append(adpop_entry)
    
    adpop_df = pd.concat(adpop_all).sort_index()
    
    if representation == 'diab':
        selector='d{}'.format(state)
        ylabel='Diabatic population'
        title='Population on diabatic state {} over time'.format(state)
    elif representation == 'adb':
        selector='a{}'.format(state)
        ylabel='Adiabatic population'
        title='Population on adiabatic state {} over time'.format(state)
    
    fig, ax = plt.subplots(1, 1)
    fig.figsize = (10, 8)
    for alpha, entry in adpop_df.groupby(level=0):
        single_alpha_data = entry.droplevel(0)
        #print(momentum)
        #print(single_momentum_data)
        ax.plot(single_alpha_data['Time'], single_alpha_data[selector], 
                label='{}'.format(alpha), linestyle='-', marker='', markersize=1)
        ax.set_title(title)
        

    print(single_alpha_data['Time'].iloc[-20])
    ax.set_xlabel('Time /fs')
    ax.set_ylabel(ylabel)
    fig.legend(title='$\\alpha_{x}$:')
    fig.show()
    
    return adpop_df

In [219]:
adpop_data = plot_adpop(representation='adb', state=2)
adpop_data['Time'].iloc[-60]

['alpha-a-0.075pi\\adp-a-0.23561944901923448', 'alpha-a-0.125pi\\adp-a-0.39269908169872414', 'alpha-a-0.25pi\\adp-a-0.7853981633974483', 'alpha-a-0.375pi\\adp-a-1.1780972450961724', 'alpha-a-0.4pi\\adp-a-1.2566370614359172', 'alpha-a0.075pi\\adp-a0.23561944901923448', 'alpha-a0.0pi\\adp-a0.0', 'alpha-a0.125pi\\adp-a0.39269908169872414', 'alpha-a0.25pi\\adp-a0.7853981633974483', 'alpha-a0.375pi\\adp-a1.1780972450961724', 'alpha-a0.4pi\\adp-a1.2566370614359172']
adp-a-0.23561944901923448
adp-a-0.39269908169872414
adp-a-0.7853981633974483
adp-a-1.1780972450961724
adp-a-1.2566370614359172
adp-a0.23561944901923448
adp-a0.0
adp-a0.39269908169872414
adp-a0.7853981633974483
adp-a1.1780972450961724
adp-a1.2566370614359172
31.5


33.0

In [220]:
#temp_e = [2.1406288 , 2.16330495, 2.23133341]
#tot_e = energies_pd['Tot.E'].to_list()
tot_e=energies_df['Tot.E']

final_transition_probabilities = []
tilts = []
for alpha, entry in adpop_data.groupby(level=0):
    single_momentum_data = entry.droplevel(0)
    print(single_momentum_data['Time'].iloc[-20])
    final_transition_probabilities.append(single_momentum_data['a1'].iloc[-20])
    tilts.append(alpha)

31.5
31.5
31.5
31.5
31.5
31.5
31.5
31.5
31.5
31.5
31.5


In [221]:
probability_vs_energy = pd.DataFrame(columns=['x tilts', 'Energy/eV', 'Transition Probability'])
probability_vs_energy['x tilts']=tilts
probability_vs_energy['Energy /Ht'] = tot_e/27.21138386
probability_vs_energy['Transition Probability'] = final_transition_probabilities


In [222]:
fig, ax = plt.subplots(1, 1)
fig.figsize = (10, 8)
ax.plot(probability_vs_energy['Energy /Ht'], probability_vs_energy['Transition Probability'],
        linestyle='--', marker='o')
ax.set_title('Energy vs transition probability')
ax.set_xlabel('Tot. Energy /Ht')
ax.set_ylabel('Transition probability')
fig.show()

In [223]:
def findKE(target_q=0):
    
    KE_x = plot_adiabatic_KE(dof='x', state=2, supress_plot=True)
    qdq_x = plot_adiabatic_qdq(dof='x', state=2, supress_plot=True)
        
    KEs=[]
    for x_momentum, x_expectation_data in qdq_x.groupby(level=0):
        KE_expectation_data = KE_x.loc[x_momentum]
        diff_array = np.abs(x_expectation_data['<x>']-target_q)
        idx = np.where(diff_array==min(diff_array))
        KEs.append(KE_expectation_data['<KE_x>'].iloc[idx])
        
    return KEs

findKE()

               Time      <x^2>
init alpha                    
-1.256637  0    0.0   3.437271
           1    1.5   2.900214
           2    3.0   2.293553
           3    4.5   1.665792
           4    6.0   1.075162
...             ...        ...
 1.256637  36  54.0  24.263965
           37  55.5  29.500804
           38  57.0  35.436417
           39  58.5  42.120081
           40  60.0  49.602611

[451 rows x 2 columns]
               Time       <x>
init alpha                   
-1.256637  0    0.0 -1.768944
           1    1.5 -1.613471
           2    3.0 -1.417634
           3    4.5 -1.181622
           4    6.0 -0.906723
...             ...       ...
 1.256637  36  54.0 -3.637228
           37  55.5 -4.010730
           38  57.0 -4.395887
           39  58.5 -4.792716
           40  60.0 -5.201350

[451 rows x 2 columns]


[7    0.245307
 Name: <KE_x>, dtype: float64,
 6    0.246784
 Name: <KE_x>, dtype: float64,
 5    0.249611
 Name: <KE_x>, dtype: float64,
 5    0.243194
 Name: <KE_x>, dtype: float64,
 5    0.240871
 Name: <KE_x>, dtype: float64,
 5    0.237441
 Name: <KE_x>, dtype: float64,
 5    0.225961
 Name: <KE_x>, dtype: float64,
 7    0.131488
 Name: <KE_x>, dtype: float64,
 5    0.221237
 Name: <KE_x>, dtype: float64,
 28    0.188395
 Name: <KE_x>, dtype: float64,
 24    0.175579
 Name: <KE_x>, dtype: float64]

In [224]:
def findmomenta(target_q=0, representation='adb'):
    
    if representation=='adb':
        px = plot_adiabatic_momenta(dof='x', state=2, supress_plot=True)
        qdq_x = plot_adiabatic_qdq(dof='x', state=2, supress_plot=True)
    elif representation=='diab':
        qdq_y = plot_qdq(dof='y', state=2, supress_plot=True)
        qdq_x = plot_qdq(dof='x', state=2, supress_plot=True)
        
    widths=[]
    for x_momentum, x_expectation_data in qdq_x.groupby(level=0):
        px_expectation_data = px.loc[x_momentum]
        diff_array = np.abs(x_expectation_data['<x>']-target_q)
        idx = np.where(diff_array==min(diff_array))
        widths.append(px_expectation_data['<p_x>'].iloc[idx])
        
    return widths

findmomenta()

               Time      <x^2>
init alpha                    
-1.256637  0    0.0   3.437271
           1    1.5   2.900214
           2    3.0   2.293553
           3    4.5   1.665792
           4    6.0   1.075162
...             ...        ...
 1.256637  36  54.0  24.263965
           37  55.5  29.500804
           38  57.0  35.436417
           39  58.5  42.120081
           40  60.0  49.602611

[451 rows x 2 columns]
               Time       <x>
init alpha                   
-1.256637  0    0.0 -1.768944
           1    1.5 -1.613471
           2    3.0 -1.417634
           3    4.5 -1.181622
           4    6.0 -0.906723
...             ...       ...
 1.256637  36  54.0 -3.637228
           37  55.5 -4.010730
           38  57.0 -4.395887
           39  58.5 -4.792716
           40  60.0 -5.201350

[451 rows x 2 columns]


[7    73.925899
 Name: <p_x>, dtype: float64,
 6    75.938088
 Name: <p_x>, dtype: float64,
 5    77.593246
 Name: <p_x>, dtype: float64,
 5    75.384772
 Name: <p_x>, dtype: float64,
 5    74.71006
 Name: <p_x>, dtype: float64,
 5    73.775904
 Name: <p_x>, dtype: float64,
 5    71.938763
 Name: <p_x>, dtype: float64,
 7    54.792837
 Name: <p_x>, dtype: float64,
 5    69.911464
 Name: <p_x>, dtype: float64,
 28   -57.383332
 Name: <p_x>, dtype: float64,
 24   -53.106995
 Name: <p_x>, dtype: float64]

In [225]:
#finds the width <dy> when the wavepacket is at a certain <q>=0
def findwidth(target_q=0, representation='adb'):
    
    if representation=='adb':
        qdq_y = plot_adiabatic_qdq(dof='y', state=2, supress_plot=True)
        qdq_x = plot_adiabatic_qdq(dof='x', state=2, supress_plot=True)
    elif representation=='diab':
        qdq_y = plot_qdq(dof='y', state=2, supress_plot=True)
        qdq_x = plot_qdq(dof='x', state=2, supress_plot=True)
        
    widths=[]
    for x_momentum, x_expectation_data in qdq_x.groupby(level=0):
        y_expectation_data = qdq_y.loc[x_momentum]
        diff_array = np.abs(x_expectation_data['<x>']-target_q)
        idx = np.where(diff_array==min(diff_array))
        widths.append(y_expectation_data['<dy>'].iloc[idx])
        
    return widths

print(findwidth(representation='adb'))

               Time     <y^2>
init alpha                   
-1.256637  0    0.0  0.236442
           1    1.5  0.235155
           2    3.0  0.231133
           3    4.5  0.224110
           4    6.0  0.213731
...             ...       ...
 1.256637  36  54.0  0.189384
           37  55.5  0.194201
           38  57.0  0.198790
           39  58.5  0.203188
           40  60.0  0.207457

[451 rows x 2 columns]
               Time           <y>
init alpha                       
-1.256637  0    0.0 -0.000000e+00
           1    1.5  3.660000e-12
           2    3.0  7.400000e-13
           3    4.5 -4.611000e-11
           4    6.0  1.133800e-10
...             ...           ...
 1.256637  36  54.0 -4.820000e-12
           37  55.5 -5.326000e-11
           38  57.0  2.077000e-11
           39  58.5  7.140000e-12
           40  60.0 -6.728000e-11

[451 rows x 2 columns]
               Time      <x^2>
init alpha                    
-1.256637  0    0.0   3.437271
           1    1.5   2.900

In [226]:
#Calculates the massey parameter
def massey(sigma, F, E_k, m):
    Massey = (2*sigma**2 * F)/((2*E_k/m)**0.5)
    return Massey

In [227]:
F=0.04
m = 15000
width_sampling_point=-0.1

#Change findwith value for width at different <x>
sigmas=np.array(list(map(lambda x: float(x), findwidth(width_sampling_point))))
x_momenta=np.array(list(map(lambda x: float(x), findmomenta(width_sampling_point))))
KEs = np.array(list(map(lambda x: float(x), findKE(width_sampling_point))))

#diab_sigmas=np.array(list(map(lambda x: float(x), findwidth(width_sampling_point, representation='diab'))))
sigmas

               Time     <y^2>
init alpha                   
-1.256637  0    0.0  0.236442
           1    1.5  0.235155
           2    3.0  0.231133
           3    4.5  0.224110
           4    6.0  0.213731
...             ...       ...
 1.256637  36  54.0  0.189384
           37  55.5  0.194201
           38  57.0  0.198790
           39  58.5  0.203188
           40  60.0  0.207457

[451 rows x 2 columns]
               Time           <y>
init alpha                       
-1.256637  0    0.0 -0.000000e+00
           1    1.5  3.660000e-12
           2    3.0  7.400000e-13
           3    4.5 -4.611000e-11
           4    6.0  1.133800e-10
...             ...           ...
 1.256637  36  54.0 -4.820000e-12
           37  55.5 -5.326000e-11
           38  57.0  2.077000e-11
           39  58.5  7.140000e-12
           40  60.0 -6.728000e-11

[451 rows x 2 columns]
               Time      <x^2>
init alpha                    
-1.256637  0    0.0   3.437271
           1    1.5   2.900

array([0.39684031, 0.41818502, 0.43663945, 0.43495735, 0.4345196 ,
       0.43395716, 0.43377626, 0.42059816, 0.45360547, 0.45169071,
       0.45094598])

In [228]:
energies_htr=tot_e/27.21138386
energies_momenta=(x_momenta**2)/m
print(energies_momenta, '\n', energies_htr)

[0.3643359  0.38443955 0.40138078 0.37885759 0.37210621 0.36285894
 0.34501238 0.24658289 0.45311246 0.41615202 0.39710705] 
 4     0.330811
3     0.330811
2     0.330811
1     0.330811
0     0.330811
6     0.330811
5     0.318273
7     0.192067
8     0.330811
9     0.330811
10    0.330811
Name: Tot.E, dtype: float64


In [229]:

massey(sigmas, F, energies_momenta, m)
probability_vs_energy

,x tilts,Energy/eV,Transition Probability,Energy /Ht
0,-1.256637,NaN,0.387916,0.330811
1,-1.178097,NaN,0.377671,0.330811
2,-0.785398,NaN,0.368057,0.330811
3,-0.392699,NaN,0.365691,0.330811
4,-0.235619,NaN,0.365080,0.330811
5,0.000000,NaN,0.364292,0.318273
6,0.235619,NaN,0.361715,0.330811
7,0.392699,NaN,0.357196,0.192067
8,0.785398,NaN,0.361519,0.330811
9,1.178097,NaN,0.358461,0.330811


In [230]:
def malhado_hynes(xi, theta):
    P = np.sqrt(1/(1+np.pi*xi*theta))
    return P

predicted_probs = malhado_hynes(massey(sigmas, F, energies_htr, m), 1)
KE_predicted_probs=malhado_hynes(massey(sigmas, F, KEs, m), 1)

In [254]:
probability_vs_energy['Massey']=massey(sigmas, F, KEs, m)
probability_vs_energy['Sigmas']=sigmas
probability_vs_energy['Momentum @ CI']=x_momenta
probability_vs_energy['K.E. @ CI']=KEs
probability_vs_energy['NATP predict']=malhado_hynes(massey(sigmas, F, KEs, m), 1)
newdf = probability_vs_energy[['x tilts', 'Momentum @ CI', 'K.E. @ CI',
                                              'Energy /Ht', 'Sigmas', 'Massey', 'Transition Probability', 'NATP predict']]
newdf = newdf.rename(columns={'Transition Probability': 'NATP', 'Energy /Ht': 'Tot. E /Ht', 'K.E. @ CI': 'K.E. @ CI /Ht'})
newdf

,x tilts,Momentum @ CI,K.E. @ CI /Ht,Tot. E /Ht,Sigmas,Massey,NATP,NATP predict
0,-1.256637,73.925899,0.245307,0.330811,0.396840,2.202911,0.387916,0.355320
1,-1.178097,75.938088,0.246784,0.330811,0.418185,2.438928,0.377671,0.339772
2,-0.785398,77.593246,0.249611,0.330811,0.436639,2.643837,0.368057,0.327810
3,-0.392699,75.384772,0.243194,0.330811,0.434957,2.657895,0.365691,0.327035
4,-0.235619,74.710060,0.240871,0.330811,0.434520,2.665309,0.365080,0.326628
5,0.000000,73.775904,0.237441,0.318273,0.433957,2.677545,0.364292,0.325960
6,0.235619,71.938763,0.225961,0.330811,0.433776,2.742429,0.361715,0.322487
7,0.392699,60.817295,0.145600,0.192067,0.420598,3.211988,0.357196,0.300275
8,0.785398,82.442022,0.267706,0.330811,0.453605,2.755171,0.361519,0.321818
9,1.178097,79.008103,0.256027,0.330811,0.451691,2.793573,0.358461,0.319826


In [246]:
clean_data=newdf
#clean_data=newdf.drop(index=[8])
avg_e = clean_data['K.E. @ CI /Ht'].mean()
avg_sigma = clean_data['Sigmas'].mean()

avg_massey = clean_data['Massey'].mean()
avg_malhado_hynes = clean_data['NATP predict'].mean()
print(avg_malhado_hynes, avg_massey)

0.32591422057773944 2.69240856771337


In [253]:
fig, ax = plt.subplots(1, 1)
fig.figsize = (10, 8)

alpha_grid=np.linspace(-1.2, 0, 100)*(180/np.pi)

ax.axhline(y=avg_malhado_hynes, label='Average MH predictions')

for i in range(len(clean_data)):
    
    #Experimental values using momentum to calculate K.E. at the CI point
    p_experiment_x = clean_data['x tilts'].iloc[i]*(180/np.pi)
    p_experiment_y = clean_data['NATP'].iloc[i]
    
    p_predictions_y = clean_data['NATP predict'].iloc[i]
    
    
    ax.plot(p_experiment_x, p_experiment_y, color='red', linestyle='', marker='o', label='Experimental data')
    ax.plot(p_experiment_x, p_predictions_y, color='skyblue', linestyle='', marker='o', label='MH predictions')
    #ax.text(p_experiment_x*1.00, p_experiment_y*1.05, momentum, fontsize=8)

ax.set_title('CI tilt vs transition probability')
ax.set_xlabel('$\\alpha_{x}$ /degrees', fontsize=14)
ax.set_ylabel('NATP')
ax.set_ylim(0,1)
handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys())
fig.show()

In [251]:
#Plot differences

fig, ax = plt.subplots(1, 1)
fig.figsize = (10, 8)

alpha_grid=np.linspace(-1.2, 0, 100)*(180/np.pi)

delta_P = clean_data['NATP']-clean_data['NATP predict']

for i in range(len(clean_data)):
    
    #Experimental values using momentum to calculate K.E. at the CI point
    p_experiment_x = clean_data['x tilts'].iloc[i]*(180/np.pi)
    p_experiment_y = delta_P.iloc[i]
    
    
    ax.plot(p_experiment_x, p_experiment_y, color='blue', linestyle='', marker='o', label='Experimental. data')
    #ax.text(p_experiment_x*1.00, p_experiment_y*1.05, momentum, fontsize=8)

ax.set_title('CI tilt vs $\\Delta P$')
ax.set_xlabel('$\\alpha_{x}$ /degrees', fontsize=14)
ax.set_ylabel('$\\Delta P$')
ax.set_ylim(0,0.1)
handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
#fig.legend(by_label.values(), by_label.keys())
fig.show()

In [69]:
massey(sigmas, F, energies_momenta, m)

array([1.38515371, 1.53952395, 1.64816008, 1.61353273, 1.74525497,
       1.63132118, 1.74525497, 6.26615278, 3.74974964])

In [72]:
r=10
X = np.linspace(-r, r, 100)
Y = np.linspace(-r, r, 100)
X, Y = np.meshgrid(X, Y)
Z = np.sqrt(X**2*Y**2)

fig = plt.figure(figsize=(15, 12))
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Z)
#fig.colorbar(Z)